# Bayesian Linear Regression (25 marks)

In [2]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, zscore
from sklearn.preprocessing import StandardScaler
import pymc as pm
import arviz as az

# Suppress the BLAS warning
# import warnings
# warnings.filterwarnings("ignore", category=UserWarning, module="pytensor.tensor.blas")